# Getting Started

This notebook will provide an introduction to leveraging the ENPA REST APIs to download and analyze data for your jurisdiction by performing the following actions:
1. <a href="#Generating-an-API-Key">Generating an API key</a>,
2. <a href="#Downloading-ENPA-Data">Downloading ENPA data</a>, and 
3. <a href="#Visualizing-the-Downloaded-Data">Visualizing the downloaded data</a>.

## Generating an API Key

This section will describe how to generate an API key within the ENPA Portal. An API key is required to download data. Note that only portal administrators may create API keys. Begin by following the steps below.

1) Navigate to your jurisdiction's *Home* page and select *Manage Access*.

![Step1](https://raw.githubusercontent.com/c19hcc/enpa-pha-jupyternotebooks/main/images/Step1.png)

2) Scroll down towards the bottom of the page until you see the *API Keys* section. Select *Add API Key*.

![Step2W](https://raw.githubusercontent.com/c19hcc/enpa-pha-jupyternotebooks/main/images/Step2W.png)

3) Give your API Key a name. Try to include an identifying feature (e.g., your name), so others know that it belongs to you. After reading the terms and conditions, select *Agree & Save*.

![Step3](https://raw.githubusercontent.com/c19hcc/enpa-pha-jupyternotebooks/main/images/Step3.png)

4) Your API credentials will appear. It is recommended that you record your *Full Key* in a text file on your computer since you will not be able to retreive it later. We will use it later in this tutorial, so feel free to paste it below (within quotation marks) in addition to saving it in a text file on your computer.

In [ ]:
myFullKey = "" # Insert your full key between the quotation marks

In [ ]:
saveKey=open("ENPA_API_Key.txt",'w')
saveKey.write(f"Full Key: {myFullKey}")
saveKey.close()

## Downloading ENPA Data

This section will describe how to download ENPA data to perform further analysis using the API key generated in the previous section. Begin by following the steps below.

1) Navigate to the *API Docs* tab within the ENPA portal, and select *Authorize*.

![Step4](https://raw.githubusercontent.com/c19hcc/enpa-pha-jupyternotebooks/main/images/Step4.png)

2) Input your *Full Key* into the dialog box and select *Authorize*. Note, the API key must be included as an `X-Api-Key` header in all requests.

![Step5W](https://raw.githubusercontent.com/c19hcc/enpa-pha-jupyternotebooks/main/images/Step5W.png)

3) Import Python's Request library and construct the necessary header to include the API key.

In [ ]:
import requests

header = {"X-Api-Key": myFullKey}

4) Specify parameters and request the desired data.
* `datasets`: Choose from one of several datasets to download :`notification`, `notificationInteractions`, `riskParameters`, `codeVerified`, `keysUploaded`, `beaconCount`, `dateExposure`, `dateExposure14d`, `dateExposureV2`, `keysUploadedVaccineStatus` , `keysUploadedVaccineStatus14d`, `keysUploadedVaccineStatusV2`, `keysUploadedWithReportType`, `keysUploadedWithReportType14d`, `periodicExposureNotification`, `periodicExposureNotification14d`, `secondaryAttack`, `secondaryAttack14d`, or `userRisk`.
* `raw`: Specify `True` to return the raw, biased data or `False` to return the debiased data.
* `start_date`: Specify a start date to download data (must be in YYYY-MM-DD format); if omitted, the default is 90 days before either the end date (if specified) or the current date.
* `end_date`: Specify an end date to download data (must be in YYYY-MM-DD format); if omitted, the default is the current date.
* `country`: Input an ISO 3166-1 code for the country (e.g., `US`).
* `state`: Input an ISO-3166-2 code for the subdivision, state, or province (e.g., `US-VA`).

In [ ]:
parameters = {
    "datasets": "notification",
    "raw": False,
    "start_date": "2022-01-01",
    "end_date": "2022-01-31",
    "country": "US",
    "state": "US-VA"
}

In [ ]:
notificationRequest = requests.get("https://api.enpa-pha.io/api/public/v2/enpa-data",
                      headers = header, params = parameters)
print("Reason: ", notificationRequest.reason, "\nStatus Code: ", notificationRequest.status_code)

A *Status Code* of 200 indicates that the data was successfully retrieved!

## Visualizing the Downloaded Data

This section will describe how to prepare and visualize data downloaded from ENPA's REST API. Begin by following the steps below.

1) Convert the downloaded data from string to dictionary form using the `json` library, so it can be easy manipulated.

In [ ]:
import json
convertedNotifications = json.loads(notificationRequest.text)

Here is what the data looks like. Let's try to visualize total number of notifications over time.

In [ ]:
convertedNotifications

2) Crawling through the dictionary structure, the total number of notifications for both iOS and Android devices can be extracted from `convertedNotifications["metricData"][i]["dataTotals"][0]['expectedValue']` where $i = 0, 3, 6, 9, \dots$. Therefore, the following function will extract the needed data.

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Displays a chart of the total number of notifications from both iOS and Android 
# devices over the specified period of time
#     param data: Notifications dataset from the ENPA REST API in JSON form
def makeTotalNotificationsChart(data):
    dates = []
    notifications = []
    for i in range(len(data["metricData"])):
        if (i % 3 == 0):
            dates.append(datetime.strptime(convertedNotifications["metricData"][i]["date"], '%Y-%m-%d'))
            notifications.append(data["metricData"][i]["dataTotals"][0]['expectedValue'])
    plt.figure(figsize=(15, 8))
    plt.rcParams.update({'font.size': 16})
    plt.plot(dates, notifications)
    plt.xticks(rotation=45)
    plt.xlabel("Date")
    plt.ylabel("Number of Notifications")
    plt.title("Total Notifications Over Time")

In [ ]:
makeTotalNotificationsChart(convertedNotifications)